In [ ]:
! pip install -q sentence-transformers datasets pylate beir ranx

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.1/134.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.init(mode="disabled")

In [ ]:
! rm -r output/
! rm -r pylate-index/

In [ ]:
import torch
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)

from pylate import evaluation, losses, models, utils

# Define model parameters for contrastive training
model_name = "rasyosef/roberta-base-amharic"  # Choose the pre-trained model you want to use as base

# Set the run name for logging and output directory
run_name = "colbert-base-amharic"
output_dir = f"output/{run_name}"

# 1. Here we define our ColBERT model. If not a ColBERT model, will add a linear layer to the base encoder.
model = models.ColBERT(
  model_name_or_path=model_name,
  document_length=256
)

# Compiling the model makes the training faster
# model = torch.compile(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(
Some weights of XLMRobertaModel were not initialized from the model checkpoint at rasyosef/roberta-base-amharic and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [ ]:
model.query_length, model.document_length

(32, 256)

In [ ]:
# Load Dataset
from datasets import load_dataset

am_relevance_dataset = load_dataset("yosefw/amharic-news-retrieval-dataset-v2-with-negatives-V2")#.select(range(4_000))
am_relevance_dataset

DatasetDict({
    train: Dataset({
        features: ['query_id', 'passage_id', 'query', 'passage', 'category', 'link', 'source_dataset', 'negative_passages'],
        num_rows: 61469
    })
    test: Dataset({
        features: ['query_id', 'passage_id', 'query', 'passage', 'category', 'link', 'source_dataset', 'negative_passages'],
        num_rows: 6832
    })
})

In [ ]:
test_passage_ids = set(am_relevance_dataset["test"]["passage_id"])
len(test_passage_ids)

6764

In [ ]:
from datasets import Dataset
from tqdm import tqdm
import random

ds_rows = []
for row in tqdm(am_relevance_dataset["train"]):
  neg_passages = row["negative_passages"]
  # neg_passages = list(filter(lambda x: x["passage_id"] not in test_passage_ids, neg_passages))
  neg_passages_filtered = neg_passages[:4] + neg_passages[-4:]

  ds_rows.append({
      "query_id": row["query_id"],
      "passage_id": row["passage_id"],
      "query": row["query"],
      "positive": row["passage"],
      "negative_1": neg_passages_filtered[0]["passage"],
      "negative_2": neg_passages_filtered[2]["passage"],
      "negative_3": neg_passages_filtered[4]["passage"],
      "negative_4": neg_passages_filtered[6]["passage"],
    })

  ds_rows.append({
      "query_id": row["query_id"],
      "passage_id": row["passage_id"],
      "query": row["query"],
      "positive": row["passage"],
      "negative_1": neg_passages_filtered[1]["passage"],
      "negative_2": neg_passages_filtered[3]["passage"],
      "negative_3": neg_passages_filtered[5]["passage"],
      "negative_4": neg_passages_filtered[7]["passage"],
    })
  # print(ds_rows)
  # break

relevance_dataset = Dataset.from_list(ds_rows).sort("query_id")#.select(range(4000))
relevance_dataset

100%|██████████| 61469/61469 [00:25<00:00, 2436.52it/s]


Dataset({
    features: ['query_id', 'passage_id', 'query', 'positive', 'negative_1', 'negative_2', 'negative_3', 'negative_4'],
    num_rows: 122938
})

In [ ]:
len(test_passage_ids.intersection(relevance_dataset["passage_id"]))

161

In [ ]:
# Split the dataset (this dataset does not have a validation set, so we split the training set)

EVAL_SIZE = 4_000

eval_dataset = relevance_dataset.select(range(EVAL_SIZE))
train_dataset = relevance_dataset.select(range(EVAL_SIZE, len(relevance_dataset)))

train_dataset[0], train_dataset

In [ ]:
eval_dataset[0], eval_dataset

In [ ]:
# Define the loss function
train_loss = losses.Contrastive(model=model)

# Initialize the evaluator
dev_evaluator = evaluation.ColBERTTripletEvaluator(
    anchors=eval_dataset["query"] * 4,
    positives=eval_dataset["positive"] * 4,
    negatives=eval_dataset["negative_1"] + eval_dataset["negative_2"] + eval_dataset["negative_3"] + eval_dataset["negative_4"],
)

In [ ]:
eval_steps = 1500
batch_size = 32  # Larger batch size often improves results, but requires more memory
num_train_epochs = 4  # Adjust based on your requirements

# Configure the training arguments (e.g., batch size, evaluation strategy, logging steps)
args = SentenceTransformerTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    run_name=run_name,  # Will be used in W&B if `wandb` is installed
    learning_rate=1e-5,
    lr_scheduler_type="linear",
    eval_strategy="steps",
    eval_steps=eval_steps,
    save_strategy="steps",
    save_steps=eval_steps,
    # save_total_limit=2,
    logging_strategy="steps",
    logging_steps=eval_steps,
)

# Initialize the trainer for the contrastive training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
    data_collator=utils.ColBERTCollator(model.tokenize),
)

In [ ]:
# Start the training process
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
1500,0.346300,0.230997,0.973250
3000,0.198200,0.189858,0.978250
4500,0.126400,0.188498,0.979063
6000,0.084300,0.182361,0.980750
7500,0.071000,0.183013,0.979875
9000,0.041200,0.186904,0.980438
10500,0.038300,0.182999,0.980438
12000,0.029400,0.183096,0.980313
13500,0.025500,0.184328,0.980688


TrainOutput(global_step=14868, training_loss=0.09913372473913012, metrics={'train_runtime': 6641.8407, 'train_samples_per_second': 71.63, 'train_steps_per_second': 2.239, 'total_flos': 0.0, 'train_loss': 0.09913372473913012, 'epoch': 4.0})

### **Evaluate**

In [ ]:
dev_evaluator(
    model=model,
    # model=models.ColBERT("/content/output/contrastive-bert-small/checkpoint-2850"),
    output_path="."
  )

{'accuracy': 0.9801875352859497}

In [ ]:
import os
from google.colab import userdata

os.environ["HF_TOKEN"] = userdata.get("HF_WRITE")

# push model to hub
trainer.model.push_to_hub("ColBERT-Amharic-Base")

### **BIER Eval**

In [ ]:
from pylate import evaluation, indexes, models, retrieve

# Step 1: Initialize the ColBERT model

# dataset = "scifact" # Choose the dataset you want to evaluate
model = models.ColBERT(
    model_name_or_path="rasyosef/ColBERT-Amharic-Base",
    device="cuda" # "cpu" or "cuda" or "mps"
)

# Step 2: Create a Voyager index
index = indexes.Voyager(
    index_folder="pylate-index",
    index_name="colbert-eval",
    override=True,  # Overwrite any existing index
)

test_dataset = am_relevance_dataset["test"]

documents = [
    {"id": pid, "text": pos}
    for pid, pos in dict(zip(
        am_relevance_dataset["test"]["passage_id"] + am_relevance_dataset["train"]["passage_id"],
        am_relevance_dataset["test"]["passage"] + am_relevance_dataset["train"]["passage"]
    )).items()
  ]

queries = [q for q in test_dataset["query"]]
qrels = {query: {pid:1} for query, pid in zip(test_dataset["query"], test_dataset["passage_id"])}

# Step 4: Encode the documents
documents_embeddings = model.encode(
    [document["text"] for document in documents],
    batch_size=32,
    is_query=False,  # Indicate that these are documents
    show_progress_bar=True,
)

# Step 5: Add document embeddings to the index
index.add_documents(
    documents_ids=[document["id"] for document in documents],
    documents_embeddings=documents_embeddings,
)

# Step 6: Encode the queries
queries_embeddings = model.encode(
    queries,
    batch_size=32,
    is_query=True,  # Indicate that these are queries
    show_progress_bar=True,
)

# Step 7: Retrieve top-k documents
retriever = retrieve.ColBERT(index=index)
scores = retriever.retrieve(
    queries_embeddings=queries_embeddings,
    k=100,  # Retrieve the top 100 matches for each query
    batch_size=128,
)

# Step 8: Evaluate the retrieval results
results = evaluation.evaluate(
    scores=scores,
    qrels=qrels,
    queries=queries,
    metrics=[f"ndcg@{k}" for k in [5, 10, 100]] # NDCG for different k values
    + ["recall@5", "recall@10", "recall@50", "recall@100"]   # Recall at k
    + ["mrr@5", "mrr@10", "mrr@100"]
)

print(results)

Encoding documents (bs=32):   0%|          | 0/2067 [00:00<?, ?it/s]

Adding documents to the index (bs=2000): 100%|██████████| 34/34 [31:21<00:00, 55.35s/it]


Encoding queries (bs=32):   0%|          | 0/214 [00:00<?, ?it/s]

Retrieving documents (bs=128): 100%|██████████| 54/54 [35:37<00:00, 39.59s/it]
/usr/local/lib/python3.11/dist-packages/ranx/metrics/ndcg.py:72: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _ndcg(qrels[i], run[i], k, rel_lvl, jarvelin)


{'ndcg@5': np.float64(0.8253545025645599), 'ndcg@10': np.float64(0.8346465001473153), 'ndcg@100': np.float64(0.8443166547375272), 'recall@5': np.float64(0.9015817223198594), 'recall@10': np.float64(0.930140597539543), 'recall@50': np.float64(0.9667545401288811), 'recall@100': np.float64(0.9745166959578208), 'mrr@5': np.float64(0.799558191759422), 'mrr@10': np.float64(0.8034251471531787), 'mrr@100': np.float64(0.8054998678863597)}


In [ ]:
# Step 8: Evaluate the retrieval results
results = evaluation.evaluate(
    scores=scores,
    qrels=qrels,
    queries=queries,
    metrics=[f"ndcg@{k}" for k in [5, 10, 100]] # NDCG for different k values
    + ["recall@5", "recall@10", "recall@50", "recall@100"]   # Recall at k
    + ["mrr@5", "mrr@10", "mrr@100"]
)

print(results)

metrics = ["ndcg@5", "ndcg@10", "ndcg@100", "mrr@5", "mrr@10", "mrr@100", "recall@5", "recall@10", "recall@50", "recall@100"]
for m in metrics:
  print(f"{m}: {results[m]}")

{'ndcg@5': np.float64(0.8253545025645599), 'ndcg@10': np.float64(0.8346465001473153), 'ndcg@100': np.float64(0.8443166547375272), 'recall@5': np.float64(0.9015817223198594), 'recall@10': np.float64(0.930140597539543), 'recall@50': np.float64(0.9667545401288811), 'recall@100': np.float64(0.9745166959578208), 'mrr@5': np.float64(0.799558191759422), 'mrr@10': np.float64(0.8034251471531787), 'mrr@100': np.float64(0.8054998678863597)}
ndcg@5: 0.8253545025645599
ndcg@10: 0.8346465001473153
ndcg@100: 0.8443166547375272
mrr@5: 0.799558191759422
mrr@10: 0.8034251471531787
mrr@100: 0.8054998678863597
recall@5: 0.9015817223198594
recall@10: 0.930140597539543
recall@50: 0.9667545401288811
recall@100: 0.9745166959578208
